In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, time
from ipywidgets import widgets, interact
from ipywidgets.embed import embed_minimal_html
from IPython.display import display

# Function to convert hhmm float to datetime.time object
def hhmm_to_time(hhmm):
    hh = int(hhmm // 100)
    mm = int(hhmm % 100)
    return time(hh, mm)

# Load the data (replace the file path with your own)
file_path = 'ROC Sept. Log Book.xlsx'  # Replace with your file path
df = pd.read_excel(file_path, sheet_name=0)

# Convert 'Start' and 'End' columns to float type
df['Start'] = pd.to_numeric(df['Start'], errors='coerce')
df['End'] = pd.to_numeric(df['End'], errors='coerce')

# Filter the data
filtered_df = df[['DATE', 'BED', 'Start', 'End']].copy()
filtered_df.dropna(subset=['DATE', 'BED', 'Start', 'End'], inplace=True)

# Convert 'DATE' column to datetime
filtered_df['DATE'] = pd.to_datetime(filtered_df['DATE'])

# Convert 'Start' and 'End' columns to datetime.time
filtered_df['Start'] = filtered_df['Start'].apply(hhmm_to_time)
filtered_df['End'] = filtered_df['End'].apply(hhmm_to_time)

# Function to plot Gantt chart
def plot_gantt_chart_with_static_time(df, selected_date):
    filtered_day_df = df[df['DATE'] == selected_date]
    filtered_day_df = filtered_day_df.sort_values(['BED', 'Start'])
    
    fig, ax = plt.subplots(figsize=(14, 8))
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
    
    xlim_start = datetime.combine(selected_date, time(6, 0))
    xlim_end = datetime.combine(selected_date, time(20, 0))
    ax.set_xlim([xlim_start, xlim_end])
    
    for idx, row in filtered_day_df.iterrows():
        start_time = datetime.combine(selected_date, row['Start'])
        end_time = datetime.combine(selected_date, row['End'])
        ax.barh(row['BED'], end_time - start_time, left=start_time, color='blue', edgecolor='black')
    
    ax.set_xlabel('Time (hh:mm)')
    ax.set_ylabel('Bed')
    ax.set_title(f'Gantt Chart for {selected_date.strftime("%Y-%m-%d")}')
    ax.invert_yaxis()
    plt.show()

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=filtered_df['DATE'].dt.date.unique(),
    description='Select Date:',
)

# Create an output widget
output = widgets.Output()

# Function to update plot based on dropdown
def update_plot(change):
    selected_date = pd.Timestamp(change['new'])
    with output:
        output.clear_output(wait=True)
        plot_gantt_chart_with_static_time(filtered_df, selected_date)

# Manually call update_plot to generate an initial plot
initial_date = pd.Timestamp(dropdown.value)
with output:
    output.clear_output(wait=True)
    plot_gantt_chart_with_static_time(filtered_df, initial_date)

# Attach update function to dropdown
dropdown.observe(update_plot, names='value')

# Explicitly display the dropdown and output widgets in the notebook
display(dropdown)
display(output)

# Embed in an HTML file
embed_minimal_html('interactive_gantt_chart.html', views=[dropdown, output], title='Interactive Gantt Chart')

Dropdown(description='Select Date:', options=(datetime.date(2023, 9, 1), datetime.date(2023, 9, 5), datetime.d…

Output()